In [5]:
!pip install -q ultralytics


In [6]:
from google.colab import files
uploaded = files.upload()
zip_name = list(uploaded.keys())[0]
print("Uploaded:", zip_name)


Saving My First Project.v1i.yolov5pytorch.zip to My First Project.v1i.yolov5pytorch (1).zip
Uploaded: My First Project.v1i.yolov5pytorch (1).zip


In [7]:
import zipfile, os, shutil

zip_path = zip_name
extract_dir = "dataset"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_dir)

print("Top-level entries in dataset/:", os.listdir(extract_dir))

# If the zip put everything inside a single folder (rare for your screenshot but safe), flatten it:
entries = os.listdir(extract_dir)
if len(entries) == 1 and os.path.isdir(os.path.join(extract_dir, entries[0])):
    inner = os.path.join(extract_dir, entries[0])
    print("Detected nested folder:", inner, "— flattening up one level")
    for name in os.listdir(inner):
        src = os.path.join(inner, name)
        dst = os.path.join(extract_dir, name)
        if os.path.exists(dst):
            # merge if folder exists
            if os.path.isdir(src):
                for f in os.listdir(src):
                    shutil.move(os.path.join(src,f), os.path.join(dst,f))
            else:
                shutil.move(src, dst)
        else:
            shutil.move(src, dst)
    shutil.rmtree(inner)
    print("Flattened. New dataset/:", os.listdir(extract_dir))

# show contents
!ls -la dataset


Top-level entries in dataset/: ['data.yaml', 'train', 'valid', 'test', 'README.dataset.txt', 'README.roboflow.txt']
total 32
drwxr-xr-x 5 root root 4096 Dec 10 18:48 .
drwxr-xr-x 1 root root 4096 Dec 10 18:54 ..
-rw-r--r-- 1 root root  328 Dec 10 18:54 data.yaml
-rw-r--r-- 1 root root  158 Dec 10 18:54 README.dataset.txt
-rw-r--r-- 1 root root 1298 Dec 10 18:54 README.roboflow.txt
drwxr-xr-x 4 root root 4096 Dec 10 18:48 test
drwxr-xr-x 4 root root 4096 Dec 10 18:48 train
drwxr-xr-x 4 root root 4096 Dec 10 18:48 valid


In [8]:
import yaml, os, glob, pprint
pp = pprint.PrettyPrinter(indent=2)

yaml_path = "dataset/data.yaml"
# find yaml if not exactly at dataset/data.yaml
if not os.path.exists(yaml_path):
    candidates = glob.glob("dataset/**/*.yaml", recursive=True) + glob.glob("dataset/**/*.yml", recursive=True)
    candidates = sorted(set(candidates))
    if candidates:
        yaml_path = candidates[0]
    else:
        raise SystemExit("No data.yaml found inside dataset/. Make sure your Roboflow ZIP included it.")

print("Using YAML:", yaml_path)
with open(yaml_path, "r") as f:
    cfg = yaml.safe_load(f)

pp.pprint(cfg)

# helper to prefer 'images' subfolder if present
def pick_images_path(base):
    cand1 = os.path.join(base, "images")
    if os.path.isdir(cand1) and any(glob.glob(os.path.join(cand1,"*.jpg")) + glob.glob(os.path.join(cand1,"*.png"))):
        return cand1
    if os.path.isdir(base) and any(glob.glob(os.path.join(base,"*.jpg")) + glob.glob(os.path.join(base,"*.png"))):
        return base
    return None

# candidates we expect from your ZIP
train_candidates = [
    os.path.join("dataset", "train", "images"),
    os.path.join("dataset", "train")
]
val_candidates = [
    os.path.join("dataset", "valid", "images"),
    os.path.join("dataset", "valid"),
    os.path.join("dataset", "val", "images"),
    os.path.join("dataset", "val")
]

train_path = None
for p in train_candidates:
    t = pick_images_path(p)
    if t:
        train_path = t
        break

val_path = None
for p in val_candidates:
    t = pick_images_path(p)
    if t:
        val_path = t
        break

# fallback: search recursively for first folder named 'train' or 'valid' that contains images
if not train_path:
    for p in glob.glob("dataset/**/train*/images", recursive=True) + glob.glob("dataset/**/train*", recursive=True):
        t = pick_images_path(p)
        if t:
            train_path = t
            break
if not val_path:
    for p in glob.glob("dataset/**/valid*/images", recursive=True) + glob.glob("dataset/**/valid*", recursive=True):
        t = pick_images_path(p)
        if t:
            val_path = t
            break

print("Detected train images folder:", train_path)
print("Detected val   images folder:", val_path)

# update cfg
changed = False
if train_path and cfg.get("train") != train_path:
    cfg["train"] = train_path; changed = True
if val_path and (cfg.get("val") != val_path and cfg.get("valid") != val_path):
    cfg["val"] = val_path; changed = True

# ensure names & nc
if "names" in cfg and "nc" not in cfg:
    cfg["nc"] = len(cfg["names"]); changed = True
elif "names" not in cfg:
    # try to read classes.txt if included
    for cand in glob.glob("dataset/**/classes.txt", recursive=True):
        with open(cand,"r") as f:
            names = [l.strip() for l in f if l.strip()]
            cfg["names"] = names
            cfg["nc"] = len(names)
            changed = True
            break

if changed:
    with open(yaml_path, "w") as f:
        yaml.safe_dump(cfg, f)
    print("Wrote updated data.yaml")
else:
    print("No change required for data.yaml")

print("\nFinal data.yaml preview:")
pp.pprint(cfg)


Using YAML: dataset/data.yaml
{ 'names': ['alcohol_bottle', 'cigarette', 'gun', 'knife', 'vape'],
  'nc': 5,
  'roboflow': { 'license': 'CC BY 4.0',
                'project': 'my-first-project-1ft8m',
                'url': 'https://universe.roboflow.com/codeguild-qruw7/my-first-project-1ft8m/dataset/1',
                'version': 1,
                'workspace': 'codeguild-qruw7'},
  'test': '../test/images',
  'train': '../train/images',
  'val': '../valid/images'}
Detected train images folder: dataset/train/images
Detected val   images folder: dataset/valid/images
Wrote updated data.yaml

Final data.yaml preview:
{ 'names': ['alcohol_bottle', 'cigarette', 'gun', 'knife', 'vape'],
  'nc': 5,
  'roboflow': { 'license': 'CC BY 4.0',
                'project': 'my-first-project-1ft8m',
                'url': 'https://universe.roboflow.com/codeguild-qruw7/my-first-project-1ft8m/dataset/1',
                'version': 1,
                'workspace': 'codeguild-qruw7'},
  'test': '../test/i

In [14]:
!ls -R dataset


dataset:
data.yaml  README.dataset.txt  README.roboflow.txt  test  train  valid

dataset/test:
images	labels

dataset/test/images:
download-1-_jpeg.rf.1e552d776ace024b4dcbbaf045c62cb1.jpg
download-4-_jpeg.rf.84a805ea964605d20bf612fc914791ae.jpg
download-4-_jpeg.rf.f67d9a7c1bc9de46b4cd2bdcf8e4aacd.jpg
download-5-_jpeg.rf.6a84d0819dcfbf5f844cf656cbaedb57.jpg
download-6-_jpeg.rf.110f1611dc256a693eaa8527e0e8b31e.jpg

dataset/test/labels:
download-1-_jpeg.rf.1e552d776ace024b4dcbbaf045c62cb1.txt
download-4-_jpeg.rf.84a805ea964605d20bf612fc914791ae.txt
download-4-_jpeg.rf.f67d9a7c1bc9de46b4cd2bdcf8e4aacd.txt
download-5-_jpeg.rf.6a84d0819dcfbf5f844cf656cbaedb57.txt
download-6-_jpeg.rf.110f1611dc256a693eaa8527e0e8b31e.txt

dataset/train:
images	labels

dataset/train/images:
download-1-_jpeg.rf.067e030aedb8679e006daa5317475faa.jpg
download-1-_jpeg.rf.3a1d42c3a1d0e6a17117e337b02556a6.jpg
download-1-_jpeg.rf.5902486eb697c8af07e3575163ea2762.jpg
download-1-_jpeg.rf.b2ed6349813087a91af55666ae4e46e6.

In [16]:
# Fix dataset/data.yaml to correct train/val image paths (safe, idempotent)
import os, yaml, pprint, glob

ROOT = "/content/dataset"   # absolute path used by Colab/Ultralytics
yaml_path = os.path.join(ROOT, "data.yaml")
pp = pprint.PrettyPrinter(indent=2)

if not os.path.exists(yaml_path):
    raise SystemExit(f"data.yaml not found at {yaml_path} - re-extract your ZIP into dataset/")

# show current content
with open(yaml_path, "r") as f:
    cfg = yaml.safe_load(f)
print("CURRENT data.yaml:")
pp.pprint(cfg)

# set correct absolute paths
train_p = os.path.join(ROOT, "train", "images")
if not os.path.exists(train_p):
    train_p = os.path.join(ROOT, "train")   # fallback if images are directly in train/
val_p   = os.path.join(ROOT, "valid", "images")
if not os.path.exists(val_p):
    val_p = os.path.join(ROOT, "valid")

cfg["train"] = train_p
cfg["val"]   = val_p

# ensure names/nc
if "names" in cfg and "nc" not in cfg:
    cfg["nc"] = len(cfg["names"])
elif "names" not in cfg:
    # try to infer names from classes.txt if present
    cand = glob.glob(os.path.join(ROOT, "**/classes.txt"), recursive=True)
    if cand:
        with open(cand[0],"r") as f:
            names = [l.strip() for l in f if l.strip()]
        if names:
            cfg["names"] = names
            cfg["nc"] = len(names)

# write back
with open(yaml_path, "w") as f:
    yaml.safe_dump(cfg, f)

print("\nWRITTEN fixed data.yaml:")
pp.pprint(cfg)

# quick verification
def quick_check(p, role):
    print(f"\n{role} -> {p}  exists={os.path.exists(p)}")
    if os.path.exists(p):
        imgs = [x for x in glob.glob(os.path.join(p,"*.*")) if x.lower().endswith((".jpg",".png",".jpeg"))]
        print(f" {role} image count (sample up to 5): {len(imgs)}")
        for im in imgs[:5]:
            print("  -", im)
        # labels folder
        labels_dir = os.path.join(os.path.dirname(p), "labels")
        print(" labels folder:", labels_dir, "exists=", os.path.isdir(labels_dir))
        if os.path.isdir(labels_dir):
            print("  label sample:", glob.glob(os.path.join(labels_dir,"*.txt"))[:3])

quick_check(cfg["train"], "TRAIN")
quick_check(cfg["val"], "VAL")

print("\nIf both TRAIN and VAL exist and show images + labels, re-run your training cell now:\n\nfrom ultralytics import YOLO\nmodel = YOLO('yolov8n.pt')\nmodel.train(data='dataset/data.yaml', epochs=50, imgsz=640, batch=16, name='moderation-yolov8n')\n")


CURRENT data.yaml:
{ 'names': ['alcohol_bottle', 'cigarette', 'gun', 'knife', 'vape'],
  'nc': 5,
  'roboflow': { 'license': 'CC BY 4.0',
                'project': 'my-first-project-1ft8m',
                'url': 'https://universe.roboflow.com/codeguild-qruw7/my-first-project-1ft8m/dataset/1',
                'version': 1,
                'workspace': 'codeguild-qruw7'},
  'test': '../test/images',
  'train': '/content/dataset/train/images',
  'val': '/content/dataset/valid/images'}

WRITTEN fixed data.yaml:
{ 'names': ['alcohol_bottle', 'cigarette', 'gun', 'knife', 'vape'],
  'nc': 5,
  'roboflow': { 'license': 'CC BY 4.0',
                'project': 'my-first-project-1ft8m',
                'url': 'https://universe.roboflow.com/codeguild-qruw7/my-first-project-1ft8m/dataset/1',
                'version': 1,
                'workspace': 'codeguild-qruw7'},
  'test': '../test/images',
  'train': '/content/dataset/train/images',
  'val': '/content/dataset/valid/images'}

TRAIN -> /cont

In [17]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")   # tiny model to start
results = model.train(
    data="dataset/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,            # lower to 8 or 4 if you get OOM
    name="moderation-yolov8n",
)



Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=moderation-yolov8n3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, p

In [18]:
!ls runs/detect
!ls runs/detect/moderation-yolov8n


moderation-yolov8n  moderation-yolov8n2  moderation-yolov8n3
args.yaml  weights


In [21]:
%%bash
echo "=== runs/detect ==="
ls -la runs/detect || true
echo "------------------"

for d in runs/detect/*; do
  if [ -d "$d/weights" ]; then
    echo "Weights in: $d"
    ls -la "$d/weights"
    echo "------------------"
  fi
done


=== runs/detect ===
total 20
drwxr-xr-x 5 root root 4096 Dec 10 19:02 .
drwxr-xr-x 3 root root 4096 Dec 10 18:55 ..
drwxr-xr-x 3 root root 4096 Dec 10 18:55 moderation-yolov8n
drwxr-xr-x 3 root root 4096 Dec 10 18:58 moderation-yolov8n2
drwxr-xr-x 3 root root 4096 Dec 10 20:22 moderation-yolov8n3
------------------
Weights in: runs/detect/moderation-yolov8n
total 8
drwxr-xr-x 2 root root 4096 Dec 10 18:55 .
drwxr-xr-x 3 root root 4096 Dec 10 18:55 ..
------------------
Weights in: runs/detect/moderation-yolov8n2
total 8
drwxr-xr-x 2 root root 4096 Dec 10 18:58 .
drwxr-xr-x 3 root root 4096 Dec 10 18:58 ..
------------------
Weights in: runs/detect/moderation-yolov8n3
total 12216
drwxr-xr-x 2 root root    4096 Dec 10 19:04 .
drwxr-xr-x 3 root root    4096 Dec 10 20:22 ..
-rw-r--r-- 1 root root 6250474 Dec 10 20:22 best.pt
-rw-r--r-- 1 root root 6250474 Dec 10 20:22 last.pt
------------------


In [22]:
from google.colab import files
files.download("runs/detect/moderation-yolov8n3/weights/best.pt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>